In [1]:
import tensorflow as tf
import numpy as np
from random import seed
from random import random
from math import exp
import sklearn as skl
from sklearn.metrics import f1_score
#from sklearn import f1_score


# Initialization 
This is where we initialize the dataset and the hidden network. 

In [15]:

#Load Dataset
inputFilenameWithPath = 'train_data.txt'
inputData = np.loadtxt(inputFilenameWithPath, delimiter=",")
n_inputs = len(inputData[0]) - 1
n_outputs = len(set([row[-1] for row in inputData]))
# Initialize a single layer neural network with n_neurons in the hidden layer
def initialize_network1(n_inputs, n_neurons , n_outputs, dropout_prob):
	network = list()
	hidden_layer1 = [{'weights' :[random() for layer in range(n_inputs)]} for i in range(n_neurons)]
	network.append(hidden_layer1)
	hidden_layer2 = [{'weights' :[random() for layer in range(n_neurons)]} for i in range(n_neurons)]
	network.append(hidden_layer2)
	output_layer = [{'weights' :[random() for layer in range(n_neurons)]} for i in range(n_outputs)]
	network.append(output_layer)
	return network

def initialize_network(n_inputs, n_neurons , n_outputs, dropout_prob):
	network = list()
	hidden_layer1 = [{'weights' :generate_layer(n_inputs),'dropped': decision(dropout_prob)} for i in range(n_neurons)]
	network.append(hidden_layer1)
	hidden_layer2 = [{'weights' :generate_layer(n_neurons),'dropped': decision(dropout_prob)} for i in range(n_neurons)]
	network.append(hidden_layer2)
	output_layer = [{'weights' :generate_layer(n_neurons),'dropped': decision(dropout_prob)} for i in range(n_outputs)]
	network.append(output_layer)
	return network

def generate_layer(nInputs):
    layer = [random() for layer in range(nInputs)]
    return layer

def decision(probability):
    return random() < probability

# Validate the input and weights initialization
Print the layer weights

In [16]:
#Testing code
network = initialize_network(n_inputs, n_neurons, n_outputs, 0.3)
for layer in network:
    for neuron in layer:
        print(neuron['dropped'])

False
False
True
True
False
False
True
False


# The product summation and sigmoid activation

In [3]:
# Calculate neuron activation for an input
def weights_input_product(weights, inputs):
	summation = 0
	for i in range(len(weights)-1):
		summation += weights[i] * inputs[i]
	return summation

In [4]:
def sigmoid(z):
	return 1.0 / (1.0 + exp(-z))

# Forward propagate

In [5]:
#Send the list of outputs for each layer
def forward_propagate(network, inputData):
    outputs = []
    inputRecord = inputData
    for layer in network: # Iterate over the layers
        layer_output = []
        i = 0
        for neuron in layer: # Iterate for all neurons
            summation = weights_input_product(neuron['weights'],inputRecord)
            if i < len(network):
                activation = sigmoid(summation)
            else:
                activation = softmax(summation)
            neuron['output'] = activation
            layer_output.append(activation)
        outputs.append(layer_output)
        inputRecord = layer_output
    return layer_output
            
        

# Test the forward propagation

In [6]:
def softmax(z):
    sum = np.sum(np.exp(z), axis=1, keepdims=True)
    return np.divide(np.exp(z),sum)

In [7]:
# Calculate the derivative of an neuron output
def transfer_derivative(output):
	return output * (1.0 - output)

def transfer_softmax_derivative(signal):
    return signal*(1-signal) + (1 - signal)*signal


def transfer_softmax_derivative1(signal):
    return np.multiply( signal, 1 - signal ) + sum(
            # handle the off-diagonal values
            - signal * np.roll( signal, i, axis = 1 )
            for i in xrange(1, signal.shape[1] )
        )

In [8]:
# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
	for i in reversed(range(len(network))):
		layer = network[i]
		errors = list()
		if i != len(network)-1:
			for j in range(len(layer)):
				error = 0.0
				for neuron in network[i + 1]:
					error += (neuron['weights'][j] * neuron['delta'])
				errors.append(error)
		else:
			for j in range(len(layer)):
				neuron = layer[j]
				errors.append(expected[j] - neuron['output'])
		for j in range(len(layer)):
			neuron = layer[j]
			if j != len(network)-1:
				neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])#Sigmoid derivative for all other layers
			else:
				neuron['delta'] = errors[j] * transfer_softmax_derivative(neuron['output'])#softmax derivative for the output layer

In [9]:
# Update network weights with error
def update_weights(network, row, l_rate):
	for i in range(len(network)):
		inputs = row[:-1]
		if i != 0:
			inputs = [neuron['output'] for neuron in network[i - 1]]
		for neuron in network[i]:
			for j in range(len(inputs)):
				neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
			neuron['weights'][-1] += l_rate * neuron['delta']

In [10]:
#Train via SGD
def train_network(network, train, l_rate, n_epoch, n_outputs,n_iterations):
	for epoch in range(n_epoch):
		sum_error = 0
		n_examples = len(train)        
		random_samples = train[np.random.choice(train.shape[0], n_iterations, replace=False), :];
		for row in random_samples:
			outputs = forward_propagate(network, row)
			expected = [0 for i in range(n_outputs)]
			#print("Expected shape",expected)
			expected[int(row[-1])] = 1
			#expected[1] = 1
			sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
			backward_propagate_error(network, expected)
			update_weights(network, row, l_rate)
		print('Epoch=%d, Loss=%.3f' % (epoch, sum_error))
    
	print("\n\nFinal Weights")
	for layer in network:
		layerWeights = []
		for neuron in layer:
			layerWeights.append(neuron['weights'])
		print layerWeights

In [11]:
def softmax(z):
    sum = np.sum(np.exp(z))
    return np.divide(np.exp(z),sum)

In [12]:
# Make a prediction with a network
p =[ 0.0871086  , 0.91817548]

def predict(network, row):
	p=outputs = forward_propagate(network, row)
	#print(outputs[0],outputs[1])
	#print(np.asarray(outputs).T)
	return outputs.index(max(outputs))


In [13]:
# Run the code now

#Do the training first.
seed(1)
inputFilenameWithPath = 'train_data.txt'
prediction_op = np.vectorize(lambda data: predict(network, data))
inputData = np.loadtxt(inputFilenameWithPath, delimiter=",")
dropout_prob = 0.3
n_inputs = len(inputData[0]) - 1
n_outputs = len(set([row[-1] for row in inputData]))
n_neurons = 3
network = initialize_network(n_inputs, n_neurons, n_outputs, dropout_prob)
sgd_learning_rate = 0.1
numberOfEpochs  = 50
numberOfExamplesPerEpoch = 100
train_network(network, inputData, sgd_learning_rate, numberOfEpochs, n_outputs,numberOfExamplesPerEpoch)
predictions = []
truth = inputData[:,2]
for row in inputData:
	prediction = predict(network, row)
	predictions.append(prediction)
f1 = skl.metrics.f1_score(truth, predictions, average='micro')  
precision = skl.metrics.precision_score(truth, predictions, average='micro')
recall = skl.metrics.recall_score(truth, predictions, average='micro')
print('\nTraining Precision=%.2f' % (precision))
print('Training Recall=%.2f' % (recall))
print('Training F1 Score=%.2f' % (f1))
    
print('\n---------------------------- Testing the predictions -------------------------')    
# Test making predictions with the network
inputFilenameWithPath = 'test_data.txt'
dataset = np.loadtxt(inputFilenameWithPath, delimiter=",")

#predictions = prediction_op(dataset[:,:2])
truth = dataset[:,2]
predictions = []
for row in dataset:
	prediction = predict(network, row)
	predictions.append(prediction)
	#print('Expected=%d, Got=%d' % (row[-1], prediction))
f1 = skl.metrics.f1_score(truth, predictions, average='micro')  
precision = skl.metrics.precision_score(truth, predictions, average='micro')
recall = skl.metrics.recall_score(truth, predictions, average='micro')
print('\nTest Precision=%.2f' % (precision))
print('Test Recall=%.2f' % (recall))
print('Test F1 Score=%.2f' % (f1))

Epoch=0, Loss=53.728
Epoch=1, Loss=50.391
Epoch=2, Loss=50.808
Epoch=3, Loss=48.347
Epoch=4, Loss=46.592
Epoch=5, Loss=45.901
Epoch=6, Loss=42.349
Epoch=7, Loss=38.601
Epoch=8, Loss=35.427
Epoch=9, Loss=29.943
Epoch=10, Loss=24.601
Epoch=11, Loss=20.064
Epoch=12, Loss=16.706
Epoch=13, Loss=12.734
Epoch=14, Loss=12.104
Epoch=15, Loss=9.215
Epoch=16, Loss=9.062
Epoch=17, Loss=7.734
Epoch=18, Loss=7.402
Epoch=19, Loss=6.593
Epoch=20, Loss=5.176
Epoch=21, Loss=4.763
Epoch=22, Loss=6.874
Epoch=23, Loss=4.940
Epoch=24, Loss=4.536
Epoch=25, Loss=6.833
Epoch=26, Loss=4.002
Epoch=27, Loss=5.330
Epoch=28, Loss=3.817
Epoch=29, Loss=3.874
Epoch=30, Loss=2.915
Epoch=31, Loss=2.171
Epoch=32, Loss=4.077
Epoch=33, Loss=3.778
Epoch=34, Loss=4.828
Epoch=35, Loss=4.249
Epoch=36, Loss=3.191
Epoch=37, Loss=3.025
Epoch=38, Loss=3.342
Epoch=39, Loss=2.284
Epoch=40, Loss=1.718
Epoch=41, Loss=4.474
Epoch=42, Loss=1.966
Epoch=43, Loss=6.067
Epoch=44, Loss=3.315
Epoch=45, Loss=3.324
Epoch=46, Loss=5.297
Epoch=47